# Analysis of the UN's World Happiness Index with machine learning

Maaike de Jong  
June 2020  
  
See the repository's [README](https://github.com/maaikedj/happiness-machine-learning/blob/master/README.md) file for background and details on the analysis and data.  

### Notebook 3: Supervised learning: linear regression
In this notebook I perform a linear regression analysis with scikit-learn, optimising the model with recursive feature elimination. 

In [ ]:
# import packages

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [ ]:
# import data

df = pd.read_csv('dfML_clean.csv')
df.head()

In [ ]:
# import df with several log transformed variables
df_log = pd.read_csv('dfML_clean_tr.csv')
df_log.head()

In [ ]:
# check correlations between variables with heatmap

corr = df.corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr)

### Linear regression  

* Build a model with 80% train, 20 % test data
* Calculate train and test R2 scores
* Average R2 scores over 1000 model runs

In the visual exploration of the variables in the previous notebook we saw that some of the features are quite heavily skewed/ not normally distributed. I'll run the model with the original clean data set as well as with the dataset with several variables log transformed, to compare which performs better. 

There were also some variables with clear outliers. Scikit-learn's RobustScaler is a suitable scaler for dealing with outliers. 

In [ ]:
# define function to run a linear regression model with 1000 cycles and calculate mean R2 socres for the test and train set

def linear_regression(df, test_size):
    
    score_list = []

    for i in range(1000):
        y = df['Happiness score']
        X = df.drop(['Happiness score'], axis=1)
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
        LM = linear_model.LinearRegression()
    
        LM.fit(X_train, y_train)
    
        y_pred = LM.predict(X_train)
        r2score_train = r2_score(y_train, y_pred)
    
        y_test_pred = LM.predict(X_test)
        r2score_test = r2_score(y_test, y_test_pred)
    
        score_list.append((r2score_train, r2score_test))
    
    score_df = pd.DataFrame(score_list, columns=('r2score_train', 'r2score_test'))
    
    return print(score_df.mean())

In [ ]:
# model with untransformed data

linear_regression(df, 0.20)

# The unscaled data performs pretty well, although there is an indication of overfitting, which is not surprising with so many variables

In [ ]:
# model with log transformed data

linear_regression(df_log, 0.20)

# The log transformed data clearly performs better than the untransformed data. The mean R2 test score is higher

In [ ]:
# Now with the data scaled with RobustScaler

scaler = RobustScaler() 

scaled = scaler.fit_transform(df_log)
df_Rscaled = pd.DataFrame(scaled)    

df_Rscaled.columns = list(df_log.columns) 

In [ ]:
# model with log transformed Robustscaled data

linear_regression(df_Rscaled, 0.20)

# This performs the same as the unscaled data

### Recursive feature elimination

Determine order of importance of features and eliminate one by one to check effect on model

In [ ]:
# run a model instance with the log transformed robust scaled data

score_list = []

y = df_Rscaled['Happiness score']
X = df_Rscaled.drop(['Happiness score'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    
LM = linear_model.LinearRegression()
    
LM.fit(X_train, y_train)
    
y_pred = LM.predict(X_train)
r2score_train = r2_score(y_train, y_pred)
    
y_test_pred = LM.predict(X_test)
r2score_test = r2_score(y_test, y_test_pred)
    
print('r2 score train = ' + str(r2score_train))
print('r2 score test = ' + str(r2score_test))

In [ ]:
# determine order of feature importance

selector = RFE(LM, n_features_to_select=1, step=1)

selector = selector.fit(X, y)

selector.ranking_

# remains the same with different runs of the model

In [ ]:
list(X.columns)

In [ ]:
# Variables in order of importance

['GDP per capita (log)', 'Life expectancy', 'Refugees % (log)', 'Renewable energy %','Drinking water services (log)',
'Population growth', 'Air pollution (log)', 'Internet use (% of population)', 'CO2 emission per capita (log)',
'Land area (log)', 'Women in parliament %', 'Compulsory education (years)', 'Protected land %', 'GDP growth (annual %)',
'Population density (log)','Gender parity index (GPI) (log)', 'Urban population']

In [ ]:
# Make a copy of the Rscaled df to use for RFE

df_RFE = df_Rscaled.copy()
df_RFE.head()

In [ ]:
# write new variation of model function returning a df with the number of features in the model and the R2 train and test scores:

def lin_reg(df, test_size):
    
    score_list = []

    for i in range(1000):
        y = df['Happiness score']
        X = df.drop(['Happiness score'], axis=1)
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
        LM = linear_model.LinearRegression()
    
        LM.fit(X_train, y_train)
    
        y_pred = LM.predict(X_train)
        r2score_train = r2_score(y_train, y_pred)
    
        y_test_pred = LM.predict(X_test)
        r2score_test = r2_score(y_test, y_test_pred)
    
        score_list.append((len(X.columns), r2score_train, r2score_test))
    
    score_df = pd.DataFrame(score_list, columns=('nr_vars', 'R2score_train', 'R2score_test'))
    score_mean = pd.DataFrame(score_df.mean()).transpose()
    
    return score_mean

In [ ]:
# create reverse list of variables to remove one by one from original df

varlist = ['Life expectancy', 'Refugees % (log)', 'Renewable energy %','Drinking water services (log)',
'Population growth', 'Air pollution (log)', 'Internet use (% of population)', 'CO2 emission per capita (log)',
'Land area (log)', 'Women in parliament %', 'Compulsory education (years)', 'Protected land %', 'GDP growth (annual %)',
'Population density (log)','Gender parity index (GPI) (log)', 'Urban population']

varlist.reverse()

varlist

In [ ]:
# create empty df with RFE results

RFE_results = pd.DataFrame(columns=['nr_vars', 'R2score_train', 'R2score_test'])

In [ ]:
# iterate through varlist to drop vars from model one by one
# run model 1000x each time and add mean R2 train and test scores to RFE results df
# This cell can take while to run!

for col in varlist:
    
    df_RFE.drop([col], axis=1, inplace = True)

    RFE_results = pd.concat([RFE_results, lin_reg(df_RFE, 0.20)], ignore_index = True)

In [ ]:
RFE_results

In [ ]:
# reformat df for plotting

RFE_melt = pd.melt(RFE_results, id_vars = ['nr_vars'], value_vars = ['R2score_train', 'R2score_test'])
RFE_melt.head()

In [ ]:
# plot train and test scores of the RFE reduced models to check the progression of the scores over the number of variables

sns.set()
sns.set_style('white')
sns.set_color_codes('pastel')

f, ax = plt.subplots(figsize=(10, 6))

sns.lineplot(x = 'nr_vars', y = 'value', hue = 'variable', palette = 'Set2', data = RFE_melt, linewidth=2)

ax.tick_params(axis='both', which='major', labelsize=16) 
ax.tick_params(axis='both', which='minor', labelsize=16)
ax.set_xticks(range(0, 17, 2))

ax.legend(loc="lower right", frameon=True, fontsize = 12)

plt.xlabel('Number of variables', fontsize=16)
plt.ylabel('R2 score', fontsize=16)
plt.suptitle('R2 train and test scores recursive feature elimination', fontsize=20)

sns.despine()

In [ ]:
# From the figure it looks like the model performs best with only the top three variables:
# 'GDP per capita (log)', 'Life expectancy', 'Refugees % (log)'
# When more variables are added the R2 test score doesn't go up anymore, while the R2 train score does go up, indicating overfitting

In [ ]:
# Compare fit of top 6 variables in a plot
# Indeed the top three show the best regression fit

x = df_log['Happiness score']

f, axes = plt.subplots(3, 3, figsize=(16, 12), sharex=True)
sns.regplot(x = x, y = df_log['GDP per capita (log)'], ax=axes[0, 0])
sns.regplot(x = x, y = df_log['Life expectancy'], ax=axes[0, 1])
sns.regplot(x = x, y = df_log['Refugees % (log)'], ax=axes[0, 2])
sns.regplot(x = x, y = df_log['Renewable energy %'], ax=axes[1, 0])
sns.regplot(x = x, y = df_log['Drinking water services (log)'], ax=axes[1, 1])
sns.regplot(x = x, y = df_log['Population growth'], ax=axes[1, 2])
sns.regplot(x = x, y = df_log['Air pollution (log)'], ax=axes[2, 0])
sns.regplot(x = x, y = df_log['Internet use (% of population)'], ax=axes[2, 1])
sns.regplot(x = x, y = df_log['CO2 emission per capita (log)'], ax=axes[2, 2])

plt.suptitle('Regression of top 6 variables with Happiness score', fontsize=24)

sns.despine()